In [1]:
import pandas as pd

In [2]:
# Read files
df_101 = pd.read_csv('300_TRANSCRIPT.csv', sep='\t')
df_301 = pd.read_csv('301_TRANSCRIPT.csv', sep='\t')
df_302 = pd.read_csv('302_TRANSCRIPT.csv', sep='\t')

In [3]:
# Concatenate
df = pd.concat([df_101, df_301, df_302])

In [6]:
df.head(5)

,start_time,stop_time,speaker,value
0,36.588,39.668,Ellie,hi i'm ellie thanks for coming in today
1,39.888,43.378,Ellie,i was created to talk to people in a safe and ...
2,43.728,48.498,Ellie,think of me as a friend i don't judge i can't ...
3,49.188,52.388,Ellie,i'm here to learn about people and would love ...
4,52.658,58.958,Ellie,i'll ask a few questions to get us started and...


In [8]:
# Combine consecutive rows with the same speaker
df_combined = df.groupby((df['speaker'] != df['speaker'].shift()).cumsum()).agg({
    'start_time': 'first',
    'stop_time': 'last',
    'speaker': 'first',
    'value': ' '.join
}).reset_index(drop=True)

In [12]:
df_combined.head(5)

,start_time,stop_time,speaker,value
0,36.588,61.378,Ellie,hi i'm ellie thanks for coming in today i was ...
1,62.328,63.178,Participant,good
2,63.798,67.528,Ellie,that's good where are you from originally
3,68.978,70.288,Participant,atlanta georgia
4,70.978,74.198,Ellie,really why'd you move to l_a


In [15]:
# Combine consecutive rows with the same speaker
df_combined = df.groupby((df['speaker'] != df['speaker'].shift()).cumsum()).agg({
    'speaker': 'first',
    'value': ' '.join
}).reset_index(drop=True)

# Initialize lists for questions and answers
questions = []
answers = []

# Separate into question and answer
for i in range(len(df_combined)):
    if df_combined.loc[i, 'speaker'] == 'Ellie':
        questions.append(df_combined.loc[i, 'value'])
        if i + 1 < len(df_combined) and df_combined.loc[i + 1, 'speaker'] == 'Participant':
            answers.append(df_combined.loc[i + 1, 'value'])
        else:
            answers.append('')
    elif df_combined.loc[i, 'speaker'] == 'Participant' and (i == 0 or df_combined.loc[i - 1, 'speaker'] != 'Ellie'):
        questions.append('')
        answers.append(df_combined.loc[i, 'value'])

# Create a new DataFrame
qa_df = pd.DataFrame({
    'question': questions,
    'answer': answers
})

In [21]:
qa_df.head(5)

,question,answer
0,hi i'm ellie thanks for coming in today i was ...,good
1,that's good where are you from originally,atlanta georgia
2,really why'd you move to l_a,um my parents are from here um
3,how do you like l_a,i love it
4,what are some things you really like about l_a,i like the weather i like the opportunities um...
